In [1]:
import os
from os import path
import pandas as pd
import tensorflow as tf
from tensorflow.keras import datasets, layers, models, optimizers
import glob
import numpy as np
import torch 
from transformers import ViTImageProcessor
from PIL import Image
from transformers import ViTForImageClassification
from torchvision.transforms import functional as F
import natsort

2023-08-03 14:12:56.387218: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
img_IDs = []
updated_pathnames = []
dir_path = os.path.join(os.getcwd() , 'TRAINING')
img_filenames= natsort.natsorted(os.listdir(dir_path))  

# get the ids from the images, where images are having three channels; omit images if channels != 3
for i, filename in enumerate(img_filenames):
#     print(i, filename)
    if filename.endswith(".jpg"):
        path_name=os.path.join(dir_path, filename)
        im = Image.open(path_name)
        im = im.resize((224, 224))  # Resize the image to (224, 224)
        imnp = np.array(im)
        if len(imnp.shape) != 3:
            continue
        img_IDs.append(filename)
        updated_pathnames.append(path_name)  #gives 9986 results
        

def get_common_strings(list1, list2):
    return list(set(list1) & set(list2))


txt_file = pd.read_csv('./training1.csv', delimiter='\t')
sub_txt_file = txt_file[['file_name', 'misogynous']]
sub_txt_file['prefix_file_name'] = sub_txt_file['file_name'].str.extract('(\d+)').astype(int)
sub_txt_file = sub_txt_file.sort_values(by='prefix_file_name', ascending=True)


txt_IDs = list(sub_txt_file.file_name)  #from the text file where we have text description 
common_strings = get_common_strings(txt_IDs, img_IDs)
print('len of common strings', len(common_strings))

common_IDs = natsort.natsorted(common_strings)
print(len(common_IDs), len(txt_IDs), len(img_IDs))


filtered_txt_file = sub_txt_file[sub_txt_file['file_name'].isin(common_IDs)].reset_index(drop=True)
filtered_txt_file

train_labels= list(filtered_txt_file['misogynous'])


train_images_path = []
for i in common_IDs:
    path_name=os.path.join(dir_path, i)
    train_images_path.append(path_name)
    
    
    
######################################## TEST IMAGES PATH ###################################
# Get IDs from test text data
test1 = pd.read_csv('Test.csv', delimiter='\t')
test_labels = pd.read_csv('test_labels.txt', 
                          delimiter='\t',
                         header=None)

test_labels.columns = ['file_name', 
                      "misogynous",
                       "shaming",
                       "stereotype",
                       "objectification",
                       "violence"]

merged_test = pd.merge(test1, test_labels, on='file_name', how='inner')

# Sort the dataframe with natural ordering of the IDs
merged_test['prefix_file_name'] = merged_test['file_name'].str.extract('(\d+)').astype(int)
# Assuming 'df' is your DataFrame
merged_test1 = merged_test.sort_values(by='prefix_file_name', ascending=True)
merged_test1

# # train = train.rename(columns={'Text Transcription': 'text'})
test2 = merged_test1.rename(columns={'Text Transcription': 'text'})
test_txt_IDs = list(test2['file_name'])



# GET IDs from test images data
test_dir_path = os.path.join(os.getcwd() , 'TEST')
test_img_IDs = natsort.natsorted(os.listdir(test_dir_path))  

# Get COmmon IDS
test_common_strings = get_common_strings(test_txt_IDs, test_img_IDs)
test_common_IDs = natsort.natsorted(test_common_strings)
print('len of common strings', len(test_common_IDs))


test_images_path = []
for i in test_common_IDs:
    path_name=os.path.join(test_dir_path, i)
    test_images_path.append(path_name)
    
# test_images_path
tst_filtered_txt_file = test2[test2['file_name'].isin(test_common_IDs)].reset_index(drop=True)
test_labels= list(tst_filtered_txt_file['misogynous'])
# test_images_path[-1], test_labels[-1]

print(len(test_images_path), len(test_labels))

len of common strings 9986
9986 10000 9986
len of common strings 1000
1000 1000


/tmp/ipykernel_26186/160188111.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_txt_file['prefix_file_name'] = sub_txt_file['file_name'].str.extract('(\d+)').astype(int)


In [3]:
def img_info(images_path, labels, batch_size=1):
    for i in range(0, len(images_path), batch_size):
#         image_batch = []
        for j in range(i, min(i+batch_size, len(images_path))):
            image_path = images_path[j]
            image = Image.open(image_path)
            image_info = { 'image': image, 'image_file_path': image_path, 'labels': labels[j] }
#             image_batch.append(image_info)

        yield image_info
    
train_images_path1 = train_images_path[0:3000]
train_labels1 = train_labels[0:3000]


total_img_info1=[]
for k in img_info(train_images_path1, train_labels1, batch_size=1):
    total_img_info1.append(k)
    

print('total image info created')    

import pickle
with open(os.path.join('18_ViTEmbeddings', '18_total_img_info1.pickle'), 'wb') as f:
    pickle.dump(total_img_info1, f)

print("Results saved as pickle object.")

del total_img_info1,  train_images_path1 , train_labels1

# ###########################################################


train_images_path2 = train_images_path[3000:6000]
train_labels2 = train_labels[3000:6000]


total_img_info2=[]
for k in img_info(train_images_path2, train_labels2, batch_size=1):
    total_img_info2.append(k)
    
print('total image info created')    

import pickle
# Save the list as a pickle object
with open(os.path.join('18_ViTEmbeddings', '18_total_img_info2.pickle'), 'wb') as f:
    pickle.dump(total_img_info2, f)

print("Results saved as pickle object.")

del total_img_info2, train_images_path2 , train_labels2 




# ########################################################

train_images_path3 = train_images_path[6000:len(train_images_path)]
train_labels3 = train_labels[6000:len(train_images_path)]


total_img_info3=[]
for k in img_info(train_images_path3, train_labels3, batch_size=1):
    total_img_info3.append(k)
    
print('total image info created')    

import pickle
# Save the list as a pickle object
with open(os.path.join('18_ViTEmbeddings', '18_total_img_info3.pickle'), 'wb') as f:
    pickle.dump(total_img_info3, f)

print("Results saved as pickle object.")

del total_img_info3, train_images_path3 , train_labels3 


import pickle

# List of pickle file names you want to load
pickle_files = ['18_total_img_info1.pickle', '18_total_img_info2.pickle', '18_total_img_info3.pickle']
# pickle_files = ['18_total_img_info1.pickle']

# Create an empty list to store the combined results
total_img_info = []

# Assuming '18_ViTEmbeddings' is the directory containing your pickle files


total_img_info = []
for file_name in pickle_files:
    fn = os.path.join('18_ViTEmbeddings', file_name)
    with open(fn, 'rb') as f:
        loaded_list = pickle.load(f)
        total_img_info.extend(loaded_list)
        
for k in img_info(test_images_path, test_labels, batch_size=1):
    total_img_info.append(k)

total image info created
Results saved as pickle object.
total image info created
Results saved as pickle object.
total image info created
Results saved as pickle object.


## Build Features

In [4]:
import torch
from torchvision.transforms.functional import resize
from transformers import ViTFeatureExtractor, ViTModel

# Load the ViT feature extractor and model
model_name = "google/vit-base-patch16-224-in21k"
feature_extractor = ViTFeatureExtractor.from_pretrained(model_name)
# model = ViTForImageClassification.from_pretrained(model_name)
model = ViTModel.from_pretrained(model_name)


# Helper function to process a batch of images and get mean-pooled embeddings
def process_batch(batch, start_idx, end_idx):
    mean_pooled_embeddings = []
    lbls = []
    for image_info in batch:
        image = image_info['image']
        lbl = image_info['labels']
        image_file_path = image_info['image_file_path']

        # Resize the image to the desired shape (224x224)
        image = resize(image, (224, 224))

        # Preprocess the image for the ViT model
        inputs = feature_extractor(images=image, return_tensors="pt")  # Make sure to return tensor
        outputs = model(**inputs)

        # Get the output embeddings (features)
        feature_embeddings = outputs.last_hidden_state.squeeze(0)

        # Apply mean pooling along the sequence length dimension
        mean_pooled_embedding = torch.mean(feature_embeddings, dim=0)

        # Append the mean-pooled embedding to the list
        mean_pooled_embeddings.append(mean_pooled_embedding)
        lbls.append(lbl)

    # Stack the embeddings to form a tensor
    mean_pooled_embeddings = torch.stack(mean_pooled_embeddings, dim=0)
    
    ################################################
    file_name = f"emb_{start_idx}_{end_idx}.pickle"
    # Save the mean-pooled embeddings as a pickle object
    save_dir = "18_ViTEmbeddings"
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    
    file_path = os.path.join(save_dir, file_name)
    
    with open(file_path, "wb") as f:
        pickle.dump(mean_pooled_embeddings, f)
        
    ################################################   
    file_name = f"label_{start_idx}_{end_idx}.pickle"
    # Save the mean-pooled embeddings as a pickle object
    save_dir = "18_ViTEmbeddings"
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    
    file_path = os.path.join(save_dir, file_name)
    
    with open(file_path, "wb") as f:
        pickle.dump(lbls, f)
    #################################################
    print(f'Batch_{start_idx}_{end_idx} Completed')

    del mean_pooled_embeddings, mean_pooled_embedding, lbls, lbl

/home/nitesh/env/dev38/python38/lib/python3.8/site-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


In [5]:
batch_size = 100
total_images = len(total_img_info)
for i in range(0, total_images, batch_size):
    batch = total_img_info[i : i + batch_size]
    process_batch(batch, i, i + batch_size)

Batch_0_100 Completed
Batch_100_200 Completed
Batch_200_300 Completed
Batch_300_400 Completed
Batch_400_500 Completed
Batch_500_600 Completed
Batch_600_700 Completed
Batch_700_800 Completed
Batch_800_900 Completed
Batch_900_1000 Completed
Batch_1000_1100 Completed
Batch_1100_1200 Completed
Batch_1200_1300 Completed
Batch_1300_1400 Completed
Batch_1400_1500 Completed
Batch_1500_1600 Completed
Batch_1600_1700 Completed
Batch_1700_1800 Completed
Batch_1800_1900 Completed
Batch_1900_2000 Completed
Batch_2000_2100 Completed
Batch_2100_2200 Completed
Batch_2200_2300 Completed
Batch_2300_2400 Completed
Batch_2400_2500 Completed
Batch_2500_2600 Completed
Batch_2600_2700 Completed
Batch_2700_2800 Completed
Batch_2800_2900 Completed
Batch_2900_3000 Completed
Batch_3000_3100 Completed
Batch_3100_3200 Completed
Batch_3200_3300 Completed
Batch_3300_3400 Completed
Batch_3400_3500 Completed
Batch_3500_3600 Completed
Batch_3600_3700 Completed
Batch_3700_3800 Completed
Batch_3800_3900 Completed
Batch_3